Making miscellaneous figures for a 2024 AGU annual meeting poster about the July 2024 NuSTAR/MaGIXS-2 co-observation campaign.

First, edit the paths in the following cell to point to the correct locations on your system.

In [ ]:
#AIA Error table - set path to location in your system.
errortab='/Users/jmdunca2/ssw/sdo/aia/response/aia_V3_error_table.txt'

#Sunpy data directory (or wherever else you store your downloaded AIA data)
sunpy_dir='/Users/jmdunca2/sunpy/data/'

#Path to top-level do-dem directory - edit for your system.
path_to_dodem = '/Users/jmdunca2/do-dem/'
from sys import path as sys_path
sys_path.append(path_to_dodem+'/dodem/')

#do-dem
import lightcurves as lc
import region_fitting as rf
import nustar_dem_prep as nu
import initial_analysis as ia
#import dodem

#Basics
import numpy as np
import importlib
import matplotlib.pyplot as plt
from astropy import units as u

#Time
import matplotlib.dates as mdates
import astropy.time
import datetime

#File wrangling
import glob
import pickle
import os
import pathlib



# from astropy.io import fits
# import nustar_pysolar as nustar
# import matplotlib.colors as colors
# from astropy.coordinates import SkyCoord
# from regions import CircleSkyRegion
import importlib
# import nustar_utilities as nuutil
import shutil


os.getcwd()

First, let's take a look at the NuSTAR lightcurves.

In [ ]:
importlib.reload(lc)
#Name your working directory
working_dir='./initial_dem/'

#Make a new working directory for prepped data/etc if it doesn't yet exist
save_path = pathlib.Path(working_dir)
if not save_path.exists():
    save_path.mkdir()
    
#Set path to obsid directory - initial pipeline should have been run already.
datapath='/Users/jmdunca2/nustar/jul-2024/21012007001/'
obsid='21012007001'
#datapath='/Users/jmdunca2/nustar/jul-2021/90710201001/'
#obsid='90710201001'


#Comment second line if you're not using this same example nustar orbit
#Edit it to include only the desired time interval (default- all times in file) once you've run this once
timerange=[]
#timerange=[datetime.datetime(2024, 7, 16, 11, 2,0), datetime.datetime(2024, 7, 16, 12, 4)]
#timerange=[datetime.datetime(2024, 7, 16, 11, 4,0), datetime.datetime(2024, 7, 16, 11, 30)]
#timerange=[datetime.datetime(2024, 7, 16, 17, 26,0), datetime.datetime(2024, 7, 16, 18, 28)]
#timerange=[datetime.datetime(2024, 7, 16, 19, 3,0), datetime.datetime(2024, 7, 16, 20, 5)]
#timerange=[datetime.datetime(2024, 7, 17, 6, 15), datetime.datetime(2024, 7, 17, 7, 15)]
timerange=[datetime.datetime(2024, 7, 17, 4, 39), datetime.datetime(2024, 7, 17, 5, 41)]

from datetime import timezone
timerange = [t.replace(tzinfo=timezone.utc) for t in timerange]

evtA = glob.glob(datapath+'/event_cl/*A06_cl.evt')
evtB = glob.glob(datapath+'/event_cl/*B06_cl.evt')
hkA  = glob.glob(datapath+'/hk/*A_fpm.hk')
hkB  = glob.glob(datapath+'/hk/*B_fpm.hk')

importlib.reload(lc)
lc.prepare_nustar_lightcurves(evtA, evtB, hkA, hkB, timebin=15, erange=[2.,4.], 
                              livetime_corr=False, save_dir=working_dir)
lc.prepare_nustar_lightcurves(evtA, evtB, hkA, hkB, timebin=15, erange=[4.,6.], 
                              livetime_corr=False, save_dir=working_dir)
lc.prepare_nustar_lightcurves(evtA, evtB, hkA, hkB, timebin=15, erange=[6.,10.], 
                              livetime_corr=False, save_dir=working_dir)

lc.plot_nustar_lightcurves(eranges = [[2.,4.],[4.,6.],[6.,10.]],
                           timerange=timerange, save_dir=working_dir)

First, make time-interval and grade specific .evt files to use to pick your regions for generating spectral data products later. To make additional copies for different regions, re-run with different directory names (working_dir) – or just copy them over (nothing is region-specific yet). 

In [ ]:
importlib.reload(nu)

working_dir='./jul-24-AR13738/'
#Make a new working directory for prepped data/etc if it doesn't yet exist
save_path = pathlib.Path(working_dir)
if not save_path.exists():
    save_path.mkdir()

#magixs2_interval = [astropy.time.Time('2024-07-16T19:03:30', scale='utc', format='isot'),
#                    astropy.time.Time('2024-07-16T19:10:00', scale='utc', format='isot')]
#time_interval=magixs2_interval
#time_interval = [astropy.time.Time('2024-07-17T06:16:00', scale='utc', format='isot'),
#                    astropy.time.Time('2024-07-17T06:32:00', scale='utc', format='isot')]
time_interval = [astropy.time.Time('2024-07-17T04:40:00', scale='utc', format='isot'),
                    astropy.time.Time('2024-07-17T05:41:00', scale='utc', format='isot')]
#time_interval = [astropy.time.Time('2024-07-16T19:03:00', scale='utc', format='isot'),
#                    astropy.time.Time('2024-07-16T20:03:00', scale='utc', format='isot')]
#time_interval= [astropy.time.Time('2024-07-16T11:03:00', scale='utc', format='isot'),
#                    astropy.time.Time('2024-07-16T12:03:00', scale='utc', format='isot')]
#time_interval= [astropy.time.Time('2024-07-16T17:27:00', scale='utc', format='isot'),
#                    astropy.time.Time('2024-07-16T17:48:00', scale='utc', format='isot')]

#Best data quality interval
#time_interval = [astropy.time.Time('2024-07-17T04:55:00', scale='utc', format='isot'),
#                    astropy.time.Time('2024-07-17T05:06:00', scale='utc', format='isot')]

#orbit 1 data interval
#time_interval= [astropy.time.Time('2024-07-16T11:04:00', scale='utc', format='isot'),
#                    astropy.time.Time('2024-07-16T11:30:00', scale='utc', format='isot')]

time = time_interval
timestring = time[0].strftime('%H-%M-%S')
stopstring = time[1].strftime('%H-%M-%S')
timestring=timestring+'_'+stopstring

# #not actually used since we aren't making spectral data products yet
regfile=path_to_dodem+'starter_region.reg'
#regfile=path_to_dodem+'/AR13743__21012007001A.reg'

clobber=False

gtifile=datapath+'event_cl/nu'+obsid+'A06_gti.fits'
nu.make_nustar_products(time_interval, 'A', gtifile, datapath, regfile, working_dir, edit_regfile=False,
                            pile_up_corr=True, clobber=clobber, path_to_dodem=path_to_dodem, dip_before_products=True)

#regfile=path_to_dodem+'/AR13743__21012007001B.reg'

gtifile=datapath+'event_cl/nu'+obsid+'B06_gti.fits'
nu.make_nustar_products(time_interval, 'B', gtifile, datapath, regfile, working_dir, edit_regfile=False,
                            pile_up_corr=True, clobber=clobber, path_to_dodem=path_to_dodem, dip_before_products=True)


In [ ]:
importlib.reload(nu)
res = nu.load_nustar(time_interval, [[2.,4.],[4.,6.],[6.,10.]], working_dir, 'A', make_nustar=True, gtifile=gtifile, datapath=datapath, 
               regfile=regfile, edit_regfile=False, compare_fpm=False, combine_fpm=True, actual_total_counts=False, nofit=False,
                   use_fit_regfile=False, clobber=False, default_err=0.2, pile_up_corr=True, special_pha='',
                   adjacent_grades=False, nuradius=150,path_to_dodem=path_to_dodem, shush=False,
                   twogauss=False, direction='', guess=[], return_for_pile_up_figure=True)

In [ ]:
engs, cnts, cnts_u=res

fig=plt.figure(figsize=(10,5))
#plt.plot(engs, cnts, label='Grade 0')
#plt.plot(engs, cnts_u, label='Unphysical Grades')
#plt.plot(engs, (cnts-0.25*cnts_u), label='Grade 0, Standard Correction')
plt.stairs(cnts[0:-1], engs, label='Grade 0', linewidth=2)
plt.stairs(cnts_u[0:-1], engs, label='Unphysical Grades', linewidth=2)
plt.stairs((cnts-0.25*cnts_u)[0:-1], engs, label='Grade 0, Standard Correction', linewidth=2)
plt.yscale('log')
plt.xlim([1.5,11])
plt.xlabel('keV', fontsize=13)
plt.ylabel('Counts', fontsize=13)
plt.legend(fontsize=13)
plt.tick_params(axis='x', which='major', labelsize=12)
plt.tick_params(axis='y', which='major', labelsize=12)
plt.savefig('AGU_pile_up.png', transparent=True)

#plt.close(fig)

We want to make a figure with images from four of the orbits where we have ~better~ (still not great) data. 

In [ ]:
from aiapy.calibrate.util import get_correction_table, get_pointing_table
from aiapy.calibrate import register, update_pointing, degradation, estimate_error
from sunpy.net import Fido
from sunpy.net import attrs as a

query = Fido.search(
        a.Instrument.aia,
        a.Physobs.intensity,
        a.Wavelength(94*u.angstrom),
        a.Time(midtime-12*u.s, midtime))
        #a.Time(time_range[0],time_range[1]))#,
        #a.Sample(sample_every)
        #)
print(query)
files = Fido.fetch(query, max_conn=1)
print(files)
try:
    amap=sunpy.map.Map(files)
    #If things keep failing on the pointing table line, it may be an issue with the actual AIA map – try another time.
    ptab = get_pointing_table(amap.date - 12 * u.h, amap.date + 12 * u.h)
except AttributeError:
    amap=sunpy.map.Map(files[0])
    #If things keep failing on the pointing table line, it may be an issue with the actual AIA map – try another time.
    ptab = get_pointing_table(amap.date - 12 * u.h, amap.date + 12 * u.h)
    

try:
    m_temp = update_pointing(amap, pointing_table=ptab)
except TypeError:
    amap.meta.pop('crpix1')
    amap.meta.pop('crpix2')
    print('CRPIX issue on ', files)
    m_temp = update_pointing(amap, pointing_table=ptab)
    
#converts lev1 map to lev1.5 map 
#(see: https://aiapy.readthedocs.io/en/stable/api/aiapy.calibrate.register.html?highlight=register)
m = register(m_temp)

In [ ]:
import nustar_utilities as nuutil
from aiapy.calibrate.util import get_correction_table, get_pointing_table
from aiapy.calibrate import register, update_pointing, degradation, estimate_error
from sunpy.net import Fido
from sunpy.net import attrs as a
import sunpy.map


midtimes=[]
for i in range(0, len(datapaths)):
    time = time_intervals[i]
    timestring = time[0].strftime('%H-%M-%S')
    stopstring = time[1].strftime('%H-%M-%S')
    timestring=timestring+'_'+stopstring
    print(timestring)
    specific_time_evt = glob.glob(working_dir+timestring+'/'+'*cl.evt') 
    evt_data, hdr = ia.return_submap(datapath=datapaths[i], fpm='A', specific_evt=specific_time_evt[0], return_evt_hdr=True)
    time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
    midtimes.append(time0 + (time1-time0).to(u.s).value/2*u.s)


aiamaps=[]
for mt in midtimes:
    query = Fido.search(
        a.Instrument.aia,
        a.Physobs.intensity,
        a.Wavelength(94*u.angstrom),
        a.Time(mt-12*u.s, mt))
        #a.Time(time_range[0],time_range[1]))#,
        #a.Sample(sample_every)
        #)
    print(query)
    files = Fido.fetch(query, max_conn=1)
    print(files)
    try:
        amap=sunpy.map.Map(files)
        #If things keep failing on the pointing table line, it may be an issue with the actual AIA map – try another time.
        ptab = get_pointing_table(amap.date - 12 * u.h, amap.date + 12 * u.h)
    except AttributeError:
        amap=sunpy.map.Map(files[0])
        #If things keep failing on the pointing table line, it may be an issue with the actual AIA map – try another time.
        ptab = get_pointing_table(amap.date - 12 * u.h, amap.date + 12 * u.h)
        
    
    try:
        m_temp = update_pointing(amap, pointing_table=ptab)
    except TypeError:
        amap.meta.pop('crpix1')
        amap.meta.pop('crpix2')
        print('CRPIX issue on ', files)
        m_temp = update_pointing(amap, pointing_table=ptab)
        
    #converts lev1 map to lev1.5 map 
    #(see: https://aiapy.readthedocs.io/en/stable/api/aiapy.calibrate.register.html?highlight=register)
    m = register(m_temp)

    aiamaps.append(m)
    

In [ ]:
from astropy.coordinates import SkyCoord, SkyOffsetFrame
from sunpy.coordinates import Helioprojective
import matplotlib.colors as colors
from scipy.ndimage.interpolation import shift

datapaths=['/Users/jmdunca2/nustar/jul-2024/21012001001/',
           '/Users/jmdunca2/nustar/jul-2024/21012005001/',
           '/Users/jmdunca2/nustar/jul-2024/21012006001/',
           '/Users/jmdunca2/nustar/jul-2024/21012007001/']

time_intervals = [[astropy.time.Time('2024-07-16T11:03:00', scale='utc', format='isot'),
                    astropy.time.Time('2024-07-16T12:03:00', scale='utc', format='isot')],
                  [astropy.time.Time('2024-07-16T17:27:00', scale='utc', format='isot'),
                    astropy.time.Time('2024-07-16T17:48:00', scale='utc', format='isot')],
                  [astropy.time.Time('2024-07-16T19:03:00', scale='utc', format='isot'),
                    astropy.time.Time('2024-07-16T20:03:00', scale='utc', format='isot')],
                   [astropy.time.Time('2024-07-17T04:40:00', scale='utc', format='isot'),
                    astropy.time.Time('2024-07-17T05:41:00', scale='utc', format='isot')]
                 ]
working_dir='./jul-24-AR13738/'


nushifts = [
    [-10,40],
    [-50,50],
    [-100,30],
    [0,-50]
]

# nushifts = [
#     [0,0],
#     [0,0],
#     [0,0],
#     [0,0]
# ]

fig = plt.figure(figsize=(22,6))

for i in range(0, len(datapaths)):
    time = time_intervals[i]
    timestring = time[0].strftime('%H-%M-%S')
    stopstring = time[1].strftime('%H-%M-%S')
    timestring=timestring+'_'+stopstring
    specific_time_evt = glob.glob(working_dir+timestring+'/'+'*cl.evt') 
    specific_time_evt.sort()
    print(specific_time_evt)
    evtA = specific_time_evt[3]
    nu_smap = ia.return_submap(datapath=datapaths[i], fpm='B', specific_evt=evtA)

    nushift = nushifts[i]
    print(nushift)
    
    #Note: axes have same scale, used axis1 for both coordinates
    xshift=nushift[0]/nu_smap.scale.axis1.value
    yshift=nushift[1]/nu_smap.scale.axis1.value
    
    #Making shifted NuSTAR submap
    shifted_nu_smap = shift(nu_smap.data, [yshift, xshift], mode='constant')
    shifted_nu_smap=sunpy.map.Map(shifted_nu_smap, nu_smap.meta)

    nu_smap=shifted_nu_smap

    aiamap=aiamaps[i]
    aiamap.plot_settings['norm'] = savenorm

    #Need to reproject a specific way if the NuSTAR FOV extends over the limb.
                
    #Make a new observer object, using the observer from the AIA map
    new_observer = aiamap.observer_coordinate
    
    #Note: for some reason, using the same data shape with a reprojection can cause most of the data to 
    #be cropped out and set to NaN values.
    #To be safe, we will use a VERY large output shape.
    #out_shape = nu_smap.data.shape
    out_shape = (1500,1500)
    
    out_ref_coord = SkyCoord(0*u.arcsec, 0*u.arcsec, obstime=new_observer.obstime,
                             frame='helioprojective', observer=new_observer,
                             rsun=nu_smap.coordinate_frame.rsun)
    
    out_header = sunpy.map.make_fitswcs_header(
        out_shape,
        out_ref_coord,
        scale=u.Quantity(nu_smap.scale)
        )

    xx, yy = 400, -200

    with Helioprojective.assume_spherical_screen(nu_smap.observer_coordinate):
        nu_reproject = nu_smap.reproject_to(out_header)

    bl=[(xx-600)*u.arcsec, (yy-600)*u.arcsec]
    tr=[(xx+800)*u.arcsec,(yy+800)*u.arcsec]

    world_coords = SkyCoord(Tx=[bl[0], tr[0]], Ty=[bl[1],tr[1]], frame=aiamap.coordinate_frame)
    apixel_coords_x, apixel_coords_y = aiamap.wcs.world_to_pixel(world_coords)
    

    ax = fig.add_subplot(1,4,(i+1), projection=aiamap)
    aiamap.plot(axes=ax, zorder=0)                    
    levels = np.array([1, 5, 10, 50, 90, 95])*u.percent 
    nu_reproject.draw_contours(levels, axes=ax, alpha=1, zorder=1, cmap='plasma')
    ax.set_xlim(apixel_coords_x)
    ax.set_ylim(apixel_coords_y) 
    ax.text(0.1,0.9, 'Contours: 1, 5, 10, 50, 90, 95%', color='white',transform=ax.transAxes, fontsize=15)

    ax.spines['bottom'].set_color('white')
    ax.spines['top'].set_color('white')
    ax.spines['left'].set_color('white')
    ax.spines['right'].set_color('white')
    ax.xaxis.label.set_color('white')
    ax.yaxis.label.set_color('white')
    ax.title.set_color('white')
    ax.set_xlabel('X', color='white')
    ax.set_ylabel('Y', color='white')
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')

plt.savefig('AGU_images.png', transparent=True)

In [ ]:
# #Make a copy to do the other AR - if running for the first time
# working_dir2='./jul-24-AR13743/'
# shutil.copytree(working_dir+timestring, working_dir2+timestring)

Now, make images in each grade, and refine the desired regions.

In [ ]:
importlib.reload(ia)

#midtime=astropy.time.Time('2024-07-16T19:06:45', scale='utc', format='isot')

# working_dir='./jul-24-AR13743/'
# regionsavename='AR13743_'
# savefigdir=working_dir+timestring
# specific_time_evt = glob.glob(working_dir+timestring+'/'+'*cl.evt') #.sort()
# specific_time_evt.sort()
# #print(specific_time_evt)

# #orbit7
# regiondict = {'radius': 200,
#               'centerx': 500*u.arcsec,
#               'centery': -150*u.arcsec}


# ia.nuevtplot(datapath=datapath, evtA=specific_time_evt[0], evtB=specific_time_evt[3],
#           savefigdir=savefigdir,
#           regiondictA=regiondict, regiondictB=regiondict,
#          regionsave=True, regionsavename=regionsavename)    
# ia.nuevtplot(datapath=datapath, evtA=specific_time_evt[1], evtB=specific_time_evt[4],
#           savefigdir=savefigdir,
#           regiondictA=regiondict, regiondictB=regiondict,
#          regionsave=True, regionsavename=regionsavename) 
# ia.nuevtplot(datapath=datapath, evtA=specific_time_evt[2], evtB=specific_time_evt[5],
#           savefigdir=savefigdir,
#           regiondictA=regiondict, regiondictB=regiondict,
#          regionsave=True, regionsavename=regionsavename)



working_dir='./jul-24-AR13738/'
regionsavename='AR13738_'
savefigdir=working_dir+timestring
specific_time_evt = glob.glob(working_dir+timestring+'/'+'*cl.evt') #.sort()
specific_time_evt.sort()

#orbit1
regiondict = {'radius': 200,
             'centerx': 1000*u.arcsec,
             'centery': -200*u.arcsec}

#orbit7
# regiondict = {'radius': 200,
#               'centerx': 1000*u.arcsec,
#               'centery': -100*u.arcsec}


ia.nuevtplot(datapath=datapath, evtA=specific_time_evt[0], evtB=specific_time_evt[3],
          savefigdir=savefigdir,
          regiondictA=regiondict, regiondictB=regiondict,
         regionsave=True, regionsavename=regionsavename)    
ia.nuevtplot(datapath=datapath, evtA=specific_time_evt[1], evtB=specific_time_evt[4],
          savefigdir=savefigdir,
          regiondictA=regiondict, regiondictB=regiondict,
         regionsave=True, regionsavename=regionsavename) 
ia.nuevtplot(datapath=datapath, evtA=specific_time_evt[2], evtB=specific_time_evt[5],
          savefigdir=savefigdir,
          regiondictA=regiondict, regiondictB=regiondict,
         regionsave=True, regionsavename=regionsavename)

Once more for just grade 0, but with AIA94 context:

In [ ]:
importlib.reload(ia)

nushift=[-50, 50]

#midtime=astropy.time.Time('2024-07-16T19:06:45', scale='utc', format='isot')

working_dir='./jul-24-AR13743/'
regionsavename='AR13743_'
savefigdir=working_dir+timestring
specific_time_evt = glob.glob(working_dir+timestring+'/'+'*cl.evt') #.sort()
specific_time_evt.sort()
#print(specific_time_evt)

#orbit7
#regiondict = {'radius': 200,
#              'centerx': 500*u.arcsec,
#              'centery': -150*u.arcsec}

#orbit6
regiondict = {'radius': 200,
              'centerx': 450*u.arcsec,
              'centery': -400*u.arcsec}
  
# m, nu_smap, aia_regiondict = ia.nuevtplot(datapath, evtA=specific_time_evt[1], evtB=specific_time_evt[4],
#           savefigdir=savefigdir, AIA94=True, input_aia=m,
#           regiondictA=regiondict, regiondictB=regiondict,
#          regionsave=True, regionsavename=regionsavename, 
#                          overlimb=True, nushift=[-50, 50]) 

# m2, nu_smap, aia_regiondict = ia.nuevtplot(datapath=datapath, evtA=specific_time_evt[1], evtB=specific_time_evt[4],
#           savefigdir=savefigdir, AIA94=True, #input_aia=m2, 
#           regiondictA=regiondict, regiondictB=regiondict,
#          regionsave=True, regionsavename=regionsavename, 
#                          overlimb=True, nushift=nushift) 

working_dir='./jul-24-AR13738/'
regionsavename='AR13738_'
savefigdir=working_dir+timestring
specific_time_evt = glob.glob(working_dir+timestring+'/'+'*cl.evt') #.sort()
specific_time_evt.sort()
#print(specific_time_evt)

#orbit1
regiondict = {'radius': 200,
             'centerx': 1000*u.arcsec,
             'centery': -200*u.arcsec}

#orbit7
#regiondict = {'radius': 200,
#              'centerx': 1000*u.arcsec,
#              'centery': -100*u.arcsec}

#orbit6
# regiondict = {'radius': 200,
#               'centerx': 900*u.arcsec,
#               'centery': -150*u.arcsec}

  
m2, nu_smap, aia_regiondict = ia.nuevtplot(datapath=datapath, evtA=specific_time_evt[1], evtB=specific_time_evt[4],
          savefigdir=savefigdir, AIA94=True, input_aia=m2, 
          regiondictA=regiondict, regiondictB=regiondict,
         regionsave=True, regionsavename=regionsavename, 
                         overlimb=True, nushift=nushift) 


In [ ]:
fig = plt.figure(figsize=(14,22))
ax = fig.add_subplot(111, projection=m2)
m2.plot()
ax.set_xlim([2500,4000])
ax.set_ylim([1000,2500])  

In [ ]:
#saving maps to make example notebook for overplot
# filename='map_t'+m.date.strftime('%y-%m-%d_%H-%M-%S')+'_94A.fits'
# m.save(filename, overwrite='True')
# filename='nustar_map_t'+nu_smap.date.strftime('%y-%m-%d_%H-%M-%S')+'.fits'
# nu_smap.save(filename, overwrite='True')

Now that we have region files for each of our desired regions, it's time to make spectral data products.

In [ ]:
fpmz=['A','B']
working_dir='./jul-24-AR13738/'
regionsavename='AR13738_'

for fpm in fpmz:
    gtifile=datapath+'event_cl/nu'+obsid+fpm+'06_gti.fits'
    regfile=regionsavename+fpm+'.reg'
    nu.make_nustar_products(time_interval, fpm, gtifile, datapath, regfile, working_dir, edit_regfile=False,
                            pile_up_corr=True, clobber=False, path_to_dodem=path_to_dodem)

In [ ]:
fpmz=['A','B']
working_dir='./jul-24-AR13743/'
regionsavename='AR13743_'

for fpm in fpmz:
    gtifile=datapath+'event_cl/nu'+obsid+fpm+'06_gti.fits'
    regfile=regionsavename+fpm+'.reg'
    nu.make_nustar_products(time_interval, fpm, gtifile, datapath, regfile, working_dir, edit_regfile=False,
                            pile_up_corr=True, clobber=False, path_to_dodem=path_to_dodem)

Now that we have spectral data products, it's time to make grade separated spectra:

In [ ]:
importlib.reload(ia)

working_dir='./jul-24-AR13738/'
fpmz=['A','B']
for fpm in fpmz:
    ia.plot_grade_spectra(working_dir, timestring, fpm)

In [ ]:
working_dir='./jul-24-AR13743/'
fpmz=['A','B']
for fpm in fpmz:
    ia.plot_grade_spectra(working_dir, timestring, fpm)